In [1]:
function g!(x, y)
    for i in 1:length(x)
        x[i] *= 3
        y[i] += x[i]
    end
    x, y
end

function h!()
    for i in 1:length(x_global)
        x_global[i] *= 3
        y_global[i] += x_global[i]
    end
end

function f0()
    x = randn(100, 100)
    y = randn(100, 100)
    x, y
end

function f1()
    x = randn(100, 100)
    y = randn(100, 100)    
    g!(x, y)
end

function f2!(x = randn(100, 100), y = randn(100, 100))
    g!(x, y)
end

function f3!()
   g!(x_global, y_global)
end

function f4!()
   h!()
end

using BenchmarkTools

x_global = randn(100, 100)
y_global = randn(100, 100)

@show VERSION
@btime g!($x_global, $y_global)
@btime h!()
@btime f0()
@btime f1()
@btime f2!()
@btime f3!()
@btime f4!()

VERSION = v"1.8.0-DEV.43"
  5.267 μs (0 allocations: 0 bytes)
  1.617 ms (78980 allocations: 1.36 MiB)
  38.900 μs (4 allocations: 156.41 KiB)
  44.600 μs (4 allocations: 156.41 KiB)
  46.100 μs (4 allocations: 156.41 KiB)
  5.283 μs (1 allocation: 32 bytes)
  1.597 ms (78980 allocations: 1.36 MiB)


In [2]:
@code_warntype h!()

MethodInstance for h!()
  from h!() in Main at In[1]:9
Arguments
  #self#::Core.Const(h!)
Locals
  @_2::Any
  i::Any
Body::Nothing
1 ─ %1  = Main.length(Main.x_global)::Any
│   %2  = (1:%1)::Any
│         (@_2 = Base.iterate(%2))
│   %4  = (@_2 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_2::Any
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Any
│   %10 = Base.getindex(Main.x_global, i)::Any
│   %11 = (%10 * 3)::Any
│         Base.setindex!(Main.x_global, %11, i)
│   %13 = Base.getindex(Main.y_global, i)::Any
│   %14 = Base.getindex(Main.x_global, i)::Any
│   %15 = (%13 + %14)::Any
│         Base.setindex!(Main.y_global, %15, i)
│         (@_2 = Base.iterate(%2, %9))
│   %18 = (@_2 === nothing)::Bool
│   %19 = Base.not_int(%18)::Bool
└──       goto #4 if not %19
3 ─       goto #2
4 ┄       return nothing



In [3]:
@code_warntype g!(x_global, y_global)

MethodInstance for g!(::Matrix{Float64}, ::Matrix{Float64})
  from g!(x, y) in Main at In[1]:1
Arguments
  #self#::Core.Const(g!)
  x::Matrix{Float64}
  y::Matrix{Float64}
Locals
  @_4::Union{Nothing, Tuple{Int64, Int64}}
  i::Int64
Body::Tuple{Matrix{Float64}, Matrix{Float64}}
1 ─ %1  = Main.length(x)::Int64
│   %2  = (1:%1)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_4::Tuple{Int64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = Base.getindex(x, i)::Float64
│   %11 = (%10 * 3)::Float64
│         Base.setindex!(x, %11, i)
│   %13 = Base.getindex(y, i)::Float64
│   %14 = Base.getindex(x, i)::Float64
│   %15 = (%13 + %14)::Float64
│         Base.setindex!(y, %15, i)
│         (@_4 = Base.iterate(%2, %9))
│   %18 = (@_4 === nothing)::Bool
│   %19 = Base.not_int(%18)::Bool
└──       g

In [4]:
@code_typed f3!()

CodeInfo(
1 ─ %1 = Main.x_global::Any
│   %2 = Main.y_global::Any
│   %3 = Main.g!(%1, %2)::Tuple{Any, Any}
└──      return %3
) => Tuple{Any, Any}

In [5]:
@code_typed g!(x_global, y_global)

CodeInfo(
1 ── %1  = Base.arraylen(x)::Int64
│    %2  = Base.sle_int(1, %1)::Bool
│    %3  = Base.ifelse(%2, %1, 0)::Int64
│    %4  = Base.slt_int(%3, 1)::Bool
└───       goto #3 if not %4
2 ──       Base.nothing::Nothing
└───       goto #4
3 ──       goto #4
4 ┄─ %9  = φ (#2 => true, #3 => false)::Bool
│    %10 = φ (#3 => 1)::Int64
│    %11 = φ (#3 => 1)::Int64
│    %12 = Base.not_int(%9)::Bool
└───       goto #10 if not %12
5 ┄─ %14 = φ (#4 => %10, #9 => %29)::Int64
│    %15 = φ (#4 => %11, #9 => %30)::Int64
│    %16 = Base.arrayref(true, x, %14)::Float64
│    %17 = Base.mul_float(%16, 3.0)::Float64
│          Base.arrayset(true, x, %17, %14)::Matrix{Float64}
│    %19 = Base.arrayref(true, y, %14)::Float64
│    %20 = Base.arrayref(true, x, %14)::Float64
│    %21 = Base.add_float(%19, %20)::Float64
│          Base.arrayset(true, y, %21, %14)::Matrix{Float64}
│    %23 = (%15 === %3)::Bool
└───       goto #7 if not %23
6 ──       Base.nothing::Nothing
└───       goto #8
7 ── %27 = Base.

In [6]:
@code_warntype f3!()

MethodInstance for f3!()
  from f3!() in Main at In[1]:32
Arguments
  #self#::Core.Const(f3!)
Body::Tuple{Any, Any}
1 ─ %1 = Main.g!(Main.x_global, Main.y_global)::Tuple{Any, Any}
└──      return %1

